In [7]:
# import packages for use in the program


import time, random, csv, pyautogui, pdb, traceback, sys, yaml, datetime
import pandas as pd
import csv
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from datetime import date
from itertools import product
from validate_email import validate_email

In [ ]:
#create webdriver.exe file
#only run this the first time you are using selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [38]:
# this cell actually runs the program-- FULL structure

# load the relevant chrome webdriver
path = r'C:\Users\Randy\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe'
driver = webdriver.Chrome(path)

# get the time for use in CSV naming.. 
now = datetime.datetime.now()
stringtime = now.strftime("%Y-%m-%d-%H-%M")

# set the csv name based on datetime
filepath = r'C:\Users\Randy\Documents\Cnidaria\jupyter_first\jobs_list_'+stringtime+'.csv'

# write the column names to the csv
J_list = ['Title', 'Company', 'Location', 'Type', 'Link', "Applied?"]
with open(filepath, 'a', encoding='utf-8', newline = "") as f:
    writer = csv.writer(f)
    writer.writerow(J_list)


# login to linkedIn
start_login()

# search jobs based on criteria and location
job_search()

# scrape jobs from an inputted number of pages... 
job_scrape(filepath)

# load the answer dictionary
ans_dict = answer_read()


C:\Users\Randy\AppData\Local\Temp/ipykernel_10412/1511717716.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


NameError: name 'job_scrape' is not defined

In [31]:
ans_dict = answer_read()


In [32]:
# testing cell
jobs_apply_click()


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/1902042876.py:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('jobs-apply-button').click()


True

In [36]:
# set up the browser--navigate to linkedin and log in -- starts from an opened webdriver

# FILL IN YOUR PASSWORD AND USERNAME
# WORKING


def start_login():
    
    with open(r'C:\authkeys\linkedin\linkedin-auth.txt') as auth:
        creds = auth.readlines()
    username = creds[0][0:18]
    password = creds[0][19:]
    
    # navigate to LinkedIn
    try:
        driver.get("https://www.linkedin.com/login")
    
    
    # login -- send password and username
        driver.find_element(By.ID, "username").send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
    # click the login button
        driver.find_element(By.CSS_SELECTOR, ".btn__primary--large").click()
    except:
        print("Login already completed or failed")
    
        


In [37]:
# this function navigates a logged-in browser to the jobs page and searches for the inputted title and location...

def job_search(keyword = "Junior Developer", loc = "Remote"):
    # set the job title and location to search for.. 
    
    driver.get("https://www.linkedin.com/jobs/")

    title = keyword
    location = loc
    
    # clear the previous search terms from the keyword and location spots
    try:
        # keyword button -- clear previous inputs
        driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[2]/div[1]/div/div/input[1]").clear()
        # location button -- clear previous inputs
        driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[2]/div[2]/div/div/input[1]").clear()
    except:
        print("nothing to clear!")
        pass

    
    # enter the keywords and locations to search for...
    try:
        driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[2]/div[1]/div/div/input[1]").send_keys(title)
        driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[2]/div[2]/div/div/input[1]").send_keys(location)
    # click on the search button
        driver.find_element(By.XPATH, "/html/body/div[5]/header/div/div/div/div[2]/button[1]").click()    
    except:
        print('search/inputs failed... ')
        pass
    # turn the radio button for easy apply on
    
    try:
        driver.maximize_window()
        time.sleep(random.uniform(1, 3))
        # easy apply button--click it!
        driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[3]/section/div/div/div/ul/li[8]/div/button").click()
    except:
        pass

In [6]:
'''the next functions deal with job-scraping
get_pages() takes user input to determine how many jobs to scrape
get_jobs() takes all the jobs on a page
job_scrape() runs the scraping process using the previous two functions.

'''

'the next functions deal with job-scraping\nget_pages() takes user input to determine how many jobs to scrape\nget_jobs() takes all the jobs on a page\njob_scrape() runs the scraping process using the previous two functions.\n\n'

In [7]:


# function to take user's input for the number of linkedin pages to scrape/apply to 

def get_pages():
    # validate that the user has actually inputted a valid integer, and once validated, return the int.
    while True:
        page_num = input("Please input an integer betweenn 1-25 for the number of pages you want to scrape and apply to: ")
        try:
            page_num = int(page_num)
            if page_num >= 1 and page_num <= 25:
                return page_num
        except:
            print("Invalid input: you must input an integer between 1-25")
            continue

In [8]:
# scrape information for jobs on the page, and write to a csv -- this occurs once jobs have already been searched
def get_jobs(filepath):
    # try to move through the list of jobs.. 
    driver.execute_script("window.scrollTo(0, 220)")
    
     # scroll to bottom and back up...
    try:
        # get list
        job_list = driver.find_elements_by_class_name('jobs-search-results__list')[0].find_elements_by_class_name('jobs-search-results__list-item')
    
        
        # move to bottom of the list
        actions = ActionChains(driver)    
        bottom = len(job_list) - 1
        actions.move_to_element(job_list[bottom]).perform()
    except:
        pass
    
    # get the full list of jobs from the page... 
    job_list = driver.find_elements_by_class_name('jobs-search-results__list')[0].find_elements_by_class_name('jobs-search-results__list-item')
    
    
    for job_tile in job_list:
            # set the variables to placeholders
            job_title, company, job_location, apply_method, link = "", "", "", "", ""
            
            # try to grab the job title // scroll to it... 
            try:
                actions = ActionChains(driver)
                actions.move_to_element(job_tile).perform()
                job_title = job_tile.find_element_by_class_name('job-card-list__title').text
                link = job_tile.find_element_by_class_name('job-card-list__title').get_attribute('href').split('?')[0]
                
            except:
                pass
            
            # try to grab the company
            try:
                company = job_tile.find_element_by_class_name('job-card-container__company-name').text
            except:
                pass
            
            # try to grab the location
            try:
                job_location = job_tile.find_element_by_class_name('job-card-container__metadata-item').text
            except:
                pass
            
            # try to grab the apply method
            try:
                apply_method = job_tile.find_element_by_class_name('job-card-container__apply-method').text
            except:
                pass
            
            # create a list of the elements found and print them to a CSV file for later use.. 
            J_list = [job_title, company, job_location, apply_method, link]
            print(J_list)
            
            #write to the CSV
            with open(filepath, 'a', encoding='utf-8', newline = "") as f:
                writer = csv.writer(f)
                writer.writerow(J_list)

In [9]:
# move through X pages of linkedin results--uses get_jobs() and a loop for pages -- occurs after search

def job_scrape(filepath):
    # iterate over the first x pages of results...
    x = get_pages()
    for i in range(1, x + 1):
        # run the job-scraping program
        get_jobs(filepath)
        # find the next page button
        text = f"[aria-label= 'Page {i + 1}']"
        
        # move to the next page--scroll down
        actions = ActionChains(driver)
        elem = driver.find_element_by_css_selector(text)
        actions.move_to_element(elem).perform()
        
        # and click the next button
        driver.find_element_by_css_selector(text).click()
        
        # wait 1-3 seconds
        time.sleep(random.uniform(1, 3))


In [11]:
def jobs_apply():
    pass

In [27]:
driver.find_element_by_class_name('jobs-apply-button')

NameError: name 'driver' is not defined

In [13]:
# functions to progress the program through the application pages..

# jobs_apply_click clicks the 'apply' button when found on the page...
# apply button will not appear if the position is closed or has been already applied to 
# returns a boolean telling the program to continue or not.. 
def jobs_apply_click():
    time.sleep(random.uniform(1, 3))
    try:
        driver.find_element_by_class_name('jobs-apply-button').click()
        return True
    except:
        print("Unable to find apply button! You might have already applied to this position")
        return False

# next_app_page tries to progress the application once started
# returns True when the application is finished... 
def next_app_page():
    next_button = driver.find_elements_by_class_name("artdeco-button--primary")
    for i in next_button:
        print(i.text)
        if i.text == "Next" or i.text == "Review":
            print("Next page!")
            try:
                i.click()
                return True
            except:
                continue
        if "submit" in i.text.lower():
            print("Submitting!")
            try:
                i.click()
                return False
            except:
                continue


In [14]:
# static dictionaries that are currently used to fill out applications
# need to improve this at some point and switch to a better data structure most likely. 


# function to read all the questions on a page for further use
def rip_text():
    a = []
    sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
    # if there are groupings, proceed
    if len(sec_grp) > 0:
        # for each grouping, iterate through group...
        for grp_pl in sec_grp:
            print(grp_pl.text.split('\n')[0])

            

In [15]:
# the following functions handle the answer dictionary...


# function to see if there is an answer, and if not, prompt the user & save the response
def check_match(question):
    question = question.lower()
    match = ans_dict.get(question)
    if match:
        return match
    else:
        answer = input(question + " : ")
        ans_dict[question] = answer
        answer_add(question, answer)
        return answer


# function to push the new response to csv
def answer_add(question, answer):
    with open("stored_answers.csv", 'a', encoding='utf-8', newline = "") as f:
        writer = csv.writer(f)
        writer.writerow([question, answer])
        
# function to read the answer csv in as a dict
def answer_read():
    with open("stored_answers.csv", 'r', encoding='utf-8', newline = "") as f:
        reader = csv.reader(f)
        dict_from_csv = {rows[0]:rows[1] for rows in reader}
        return dict_from_csv


In [41]:
# Functions dealing with radio buttons

def radio_answer(element):
    # find the text from the element
    header = element.text.split('\n')[0].lower()
    print(header)
    # find anything that's a radio element
    radios = element.find_elements_by_class_name("fb-radio")
    # find the labels... 
    options = element.find_elements_by_tag_name('label')      
    match = ans_dict.get(header.lower())
    print(header.lower())
    if match:
        for i in options:
            try:
                if match == i.text:
                    i.click()
            except:
                continue
    else:
        try:
            ans = check_match(header.lower)
            for i in options:
                try:
                    if ans == i.text:
                        i.click()
                except:
                    pass
        except:
            pass
fill_up()
    

How many years of work experience do you have using Truck Driving?
text input found
how many years of work experience do you have using truck driving?
Are you willing to undergo a background check, in accordance with local law/regulations?
rad found
are you willing to undergo a background check, in accordance with local law/regulations?
are you willing to undergo a background check, in accordance with local law/regulations?
Are you willing to take a drug test, in accordance with local law/regulations?
rad found
are you willing to take a drug test, in accordance with local law/regulations?
are you willing to take a drug test, in accordance with local law/regulations?


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/3445058842.py:10: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


In [17]:
# functions dealing with drop-downs and answering them...

def select_dropdown(element, text):
    select = Select(element)
    select.select_by_visible_text(text)
    
def find_drop(element):
    drop = element.find_element_by_class_name('fb-dropdown__select')
    try:
        question_text = element.text.split("\n")[0].lower()
        match = ans_dict.get(question_text).title()
        select_dropdown(drop, match)
    except:
        try: 
            ans = check_match(question_text)
            select_dropdown(drop, ans)
        except:
            print("DD failed!")
            pass

In [50]:
# functions for entering text answers.. 

# given an element, answer in the the text field

def text_answer(element):
    txt_field = element.find_element_by_class_name('fb-single-line-text__input')
    question_text = element.text.split("\n")[0].lower()
    print(question_text)
    match = ans_dict.get(question_text)
    if match:
        txt_field.clear()
        txt_field.send_keys(match)
    else:
        txt_field.clear()
        new_ans = check_match(question_text)
        txt_field.send_keys(new_ans)
sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
sec_level = sec_grp[0].find_elements_by_class_name('jobs-easy-apply-form-element')
text_answer(sec_level[0])

C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/326656384.py:17: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


how many years of work experience do you have using truck driving?
how many years of work experience do you have using truck driving? : 3


In [49]:
ans_dict


{'first name': 'Matthew',
 'last name': 'Warden',
 'phone country code': 'united states (+1)',
 'mobile phone number': '2153503519',
 'email address': 'mwarden78@gmail.com',
 'do you have 24+ months of cdl class a driving experience?': 'Yes',
 'have you completed the following level of education: high school diploma?': 'No',
 'are you willing to undergo a background check, in accordance with local law/regulations?': 'Yes',
 'are you willing to take a drug test, in accordance with local law/regulations?': 'Yes',
 "are you comfortable commuting to this job's location?": 'Yes',
 'are you comfortable working in a hybrid setting?': 'Yes',
 'have you ever worked for linkedin or microsoft or any of its affiliates in any capacity?': 'No',
 'have you ever worked for deloitte & touche, curach, ingeus, str, intelligent energy, or any other deloitte affiliates?': 'No',
 'are you legally authorized to work in the united states?': 'Yes',
 'will you now or in the future require sponsorship for employ

In [214]:
# determine the type of the element and run the according function to try and answer it... 

def det_q_type(sing_elem):
     # if there are radios... 
    if len(sing_elem.find_elements_by_class_name('fb-radio'))>0:
        print("rad found")
        try: 
            radio_answer(sing_elem)
        except:
            pass
    # if there are single text inputs
    elif len(sing_elem.find_elements_by_class_name('fb-single-line-text__input'))>0:
        print("text input found")
        try:
            text_answer(sing_elem)
        except:
            pass
    # if there are drop down inputs
    elif len(sing_elem.find_elements_by_class_name('fb-dropdown__select'))>0:
        print("DD input found")
        try:
            find_drop(sing_elem)
        except:
            pass
    elif len(sing_elem.find_elements_by_class_name('artdeco-typeahead__input')):
        print("City Input!")
        try:
            autofill_handler(sing_elem)
        except:
            pass
    
    elif len(sing_elem.find_elements_by_class_name('fb-checkboxes')) >0:
        print("checkbox found!!")
        try:
            checkbox_handler(sing_elem)
            print("NO FAIL")
        except:
            print("fail")
            pass
        

In [218]:
fill_up()

BY CHECKING THIS BOX, YOU WILL DECLARE THAT YOU READ AND UNDERSTAND THE HARSCO CORPORATION PRIVACY POLICY
checkbox found!!
by checking this box, you will declare that you read and understand the harsco corporation privacy policy
I Agree Terms & Conditions
NO FAIL


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/3445058842.py:10: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


In [206]:
sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
sec_level = sec_grp[0].find_elements_by_class_name('jobs-easy-apply-form-element')

elem = sec_level[0]
class_n = elem.find_elements_by_class_name('fb-checkboxes')
tag_n = elem.find_elements_by_tag_name('label')

    
        
checkbox_handler(elem)
sec_level

by checking this box, you will declare that you read and understand the harsco corporation privacy policy
I Agree Terms & Conditions


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/218804017.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


[<selenium.webdriver.remote.webelement.WebElement (session="280a90187c46335121febf3bef45bbca", element="e33ccf35-e889-4a26-80ed-d6208fd940a4")>]

In [199]:
sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
sec_level = sec_grp[0].find_elements_by_class_name('jobs-easy-apply-form-element')


def checkbox_handler(element):
    q_text = element.text.split('\n')[0].lower()
    options = element.find_elements_by_class_name('fb-checkboxes')
    print(q_text)
    checkboxes = element.find_elements_by_tag_name('label')
    if "privacy" in q_text.lower():
        for i in checkboxes:
            print(i.text)
            if "i agree" in i.text.lower() or "yes" in i.text.lower() or 'i understand' in i.text.lower():
                i.click()
            else:
                pass
    else:
        match = get_answer(q_text)
        if match:
            for i in checkboxes:
                if match in i.text:
                    i.click()
                else:
                    pass


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/1210095044.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


In [77]:
def autofill_handler(element):
    q_text = element.text.split('\n')[0].lower()
    print(q_text)
    # will default to answering the location, as this is the most common question.. 
    if "city" in q_text:
        try:
            # will find the input box, clear it, and input the default city
            elem_2 = element.find_element_by_class_name('artdeco-typeahead__input')
            elem_2.clear()
            loc = ans_dict.get('city')
            elem_2.send_keys(loc)
            time.sleep(1)
        except:
            pass
        try:
            driver.find_element_by_class_name('artdeco-typeahead__result').click()
        except:
            pass
    else:
        try:
            elem_2 = element.find_element_by_class_name('artdeco-typeahead__input')
            elem_2.clear()
            ans = check_match(q_text)
            elem_2.send_keys(ans)
            time.sleep(1)
            driver.find_element_by_class_name('artdeco-typeahead__result').click()
        except:
            pass
        
sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
sec_level = sec_grp[0].find_elements_by_class_name('jobs-easy-apply-form-element')
#autofill_handler(sec_level[0])

C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/424182060.py:30: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


In [150]:
#sec_level[0].find_elements_by_class_name('checkbox-urn')
#sec_level[0].find_elements_by_class_name('fb-checkboxes')
det_q_type(sec_level[0])

checkbox found!!


In [64]:
# function to upload files based on the job title--can select different resumes and cover letters based on input
def file_choice(Title = "Driver"):
        # tries to find the most common job titles applying to and use the relevant resume
    if "business analyst" in Title.lower():
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Business_Analyst\MW_Resume.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Business_Analyst\Linguistics_Persian_Gloss.pdf"
    elif "data analyst" in Title.lower():
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Data_Analyst\MW_Resume.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Data_Analyst\Linguistics_Persian_Gloss.pdf"
    elif "data integration" in Title.lower():
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Data_Integrations\MW_Resume.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Data_Integrations\Linguistics_Persian_Gloss.pdf"
    elif "engineer" in Title.lower():
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Engineer\MW_Resume.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Engineer\Linguistics_Persian_Gloss.pdf"
    elif "driver" in Title.lower():
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Driver\MW_Resume_DRIVER.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Driver\Linguistics_Persian_Gloss.pdf"
    else:
        r_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\MW_Resume.pdf"
        c_dir = r"C:\Users\Matt Warden\Documents\March_22_Jobs\Data_Analyst\Linguistics_Persian_Gloss.pdf"
    return r_dir, c_dir




# function to actually upload the files if input buttons are detected.. 
def upload_files(Title = "Driver"):
    try:
        # remove previously-attached files.. 
        r_list = driver.find_elements_by_css_selector("[aria-label='Remove uploaded document']")
        for i in r_list:
            i.click()
    except:
        pass
    
    file_upload_elements = (By.CSS_SELECTOR, "input[name='file']")
    input_buttons = driver.find_elements(file_upload_elements[0], file_upload_elements[1])
    if len(input_buttons) > 0:
        # tries to find the most common job titles applying to and use the relevant resume
        r_dir, c_dir = file_choice(Title)

        try:
            file_upload_elements = (By.CSS_SELECTOR, "input[name='file']")
            input_buttons = driver.find_elements(file_upload_elements[0], file_upload_elements[1])
            for upload_button in input_buttons:
                upload_type = upload_button.find_element(By.XPATH, "..").find_element(By.XPATH, "preceding-sibling::*")
                if 'resume' in upload_type.text.lower():
                    print("Resume detected!")
                    upload_button.send_keys(r_dir)
                elif 'cover' in upload_type.text.lower():
                    print("CL detected!")
                    upload_button.send_keys(c_dir)
        except:
            pass
    else:
        pass

In [172]:
sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
elem_pls = sec_grp[0].find_elements_by_class_name('jobs-easy-apply-form-element')
det_q_type(elem_pls[0])


checkbox found!!


C:\Users\MATTWA~1\AppData\Local\Temp/ipykernel_26808/2719191051.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')


In [23]:
# function to actually fill in the page

def fill_up():
    file_upload_elements = (By.CSS_SELECTOR, "input[name='file']")
    input_buttons = driver.find_elements(file_upload_elements[0], file_upload_elements[1])
    if len(input_buttons) > 0:
        upload_files("Driver")
    
    # list all the groupings
    sec_grp = driver.find_elements_by_class_name('jobs-easy-apply-form-section__grouping')
    # if there are groupings, proceed
    if len(sec_grp) > 0:
        # for each grouping, iterate through group...
        for grp_pl in sec_grp:
            try:
                print(grp_pl.text.split('\n')[0])
                # find all the apply elements in the group... 
                elem_pls = grp_pl.find_elements_by_class_name('jobs-easy-apply-form-element')
                # for each element in the group, identify the type and try to answer/fill in 
                for sing_elem in elem_pls:
                    det_q_type(sing_elem)
            except:
                continue

In [59]:
def apply_function():
    switch = True
    timeout = time.time() + 45   # 5 minutes from now
    while switch:
        try: 
            fill_up()
            switch = next_app_page()
            if time.time() > timeout:
                switch = False
                return "Failure"
        except:
            continue
    return "Success"

In [73]:
def job_apply_one():
    if jobs_apply_click() == True:
        success = apply_function()
    return success


In [68]:
filename = r"\jobs_list_2022-03-28-18-55.csv"

In [54]:
filepath = r"C:\Users\Matt Warden\My Python Stuff\13-Web-Scraping"

In [55]:
title = "PLACEHOLDER"

In [76]:
def jobs_list():
    # read the csv of jobs from the scraping
    df_jobs = pd.read_csv(filepath + filename, encoding = "utf-8")
    df_jobs['Applied?']='NO'
    
    for i in range(0, 2):
        if(df_jobs["Applied?"][i] == "NO"):
            driver.get(df_jobs["Link"][i])
            success = job_apply_one()
            if success == "Success":
                df_jobs['Applied?'][i] == "YES"
            time.sleep(random.uniform(5,10))
    return df_jobs

In [69]:
df_jobs = pd.read_csv(filepath + filename, encoding = "utf-8")
df_jobs['Applied?']='NO'

,Title,Company,Location,Type,Link,Applied?
0,Remote Environmental Driver Specialist III – S...,Harsco Corporation,"Las Vegas, NV",Easy Apply,https://www.linkedin.com/jobs/view/2357229513/,NO
1,"Manager, Driver Onboarding and Conversion Ops,...",DoorDash,United States,Easy Apply,https://www.linkedin.com/jobs/view/2964665872/,NO
2,"REMOTE Embedded Software Developer - BSP, Devi...",CyberCoders,"Phoenix, AZ",Easy Apply,https://www.linkedin.com/jobs/view/2987747518/,NO
3,Driver Recruiter,KGTiger,United States,Easy Apply,https://www.linkedin.com/jobs/view/2964605330/,NO
4,Driver Engagement Agent,The Salvation Army Orange County,United States,Easy Apply,https://www.linkedin.com/jobs/view/2986804866/,NO
5,Product Specialist,Vyne Group,United States,Easy Apply,https://www.linkedin.com/jobs/view/2994975412/,NO
6,CDL Driver,Crawford Thomas Recruiting,"Chicopee, MA",Easy Apply,https://www.linkedin.com/jobs/view/2987039748/,NO
7,CDL Class A OTR Driver,The Citizens Group,United States,Easy Apply,https://www.linkedin.com/jobs/view/2939902996/,NO
8,"Driver Recruiter - Top Earners make $80,000+",Roehl Transport,Greater Chicago Area,Easy Apply,https://www.linkedin.com/jobs/view/2973701246/,NO
9,Truck Driver (Independent Contractor),"WEL Companies, Inc","Illinois, United States",Easy Apply,https://www.linkedin.com/jobs/view/2902778316/,NO
